# This Python script was used to generate the machine learning models and figures used in the paper in the ROBIN RNA binders vs BindingDB classification. 

## Software License:

MIT License

Copyright (c) 2022 Yazdani et al. 

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

## Jupyter notebook within the Anaconda platform was used in writing the following script. Python version 3.8.2 was used as the coding language. 
$\;\;\;\;\;\;$



![Python logo](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/300px-Python-logo-notext.svg.png)

$\;\;\;\;\;\;$

In [ ]:
# Check Python version
from platform import python_version
print("Python version:")
print(python_version())

## macOS Catalina version 10.15.7 was used when running this code on the Anaconda platform. 

## This code was written with the following package versions. Please install the following packages with the mentioned versions for consistency of results.

### pandas --> version 1.4.1

In [ ]:
# Check pandas version
import pandas as pd
print("pandas version:")
print(pd.__version__)

### numpy --> version 1.20.3

In [ ]:
# Check numpy version
import numpy as np
print("numpy version:")
print(np.__version__)

### matplotlib --> version 3.4.3

In [ ]:
# Check matplotlib version
import matplotlib as plt
print("matplotlib version:")
print(plt.__version__)

### seaborn --> version 0.11.2

In [ ]:
# Check seaborn version
import seaborn as sns
print("seaborn version:")
print(sns.__version__)

### sklearn --> version 1.0.2

In [ ]:
# Check sklearn version
import sklearn
print("sklearn version:")
print(sklearn.__version__)

### tensorflow --> version 2.3.1

In [ ]:
# Check tensorflow version
import tensorflow as tf
print("tensorflow version:")
print(tf.__version__)

### keras --> version 2.4.0

In [ ]:
# Check keras version
print("keras version:")
print(tf.keras.__version__)

### shap --> version 0.37.0

In [ ]:
# Check shap version
import shap
print("shap version:")
print(shap.__version__)

In [ ]:
# Remove at the end
# %load_ext pycodestyle_magic
# %pycodestyle_on
# %pycodestyle_off

# Classification of ROBIN RNA binders and BindingDB protein binders

## a) Classification of ROBIN RNA binders and BindingDB protein binders (No Augmentation)

In [ ]:
# Import required packages for data processing
import pandas as pd
import numpy as np
import os

# Gets current directory of the jupyter notebook to use for directories of different files
folder_dir = os.getcwd()[:-10]

# Read in ROBIN RNA binders and BindingDB protein binders
df_ROBIN_RNA = pd.read_csv(folder_dir + "/data/Mordred_files/Mordred_ROBIN_RNA_Binder_3D.csv")
df_BindingDB = pd.read_csv(folder_dir + "/data/Mordred_files/Mordred_BindingDB_3D.csv")

# Remove the name column
df_ROBIN_RNA = df_ROBIN_RNA.drop('name', axis=1)
df_BindingDB = df_BindingDB.drop('name', axis=1)

# Delete duplicate rows that might still remain in the feature sets
df_ROBIN_RNA.drop_duplicates(keep="first", inplace=True)
df_BindingDB.drop_duplicates(keep="first", inplace=True)

# ________________________________________________

# Report the number of compounds in each library
print("Number of compounds in the BindingDB protein binding library:")
print(df_BindingDB.shape[0])
print()

print("Number of compounds in the ROBIN RNA Binding library:")
print(df_ROBIN_RNA.shape[0])

In [ ]:
# Combine the two dataframes and conduct further processing 

"""
This cell might take a long time to run. This code does not have to be rerun if 
ROBIN_RNA_vs_BindingDB.csv is already saved in the ML_CSVs folder. 
"""

# Add libary names to the dataframes
df_ROBIN_RNA["Library"] = "RNA Binding"
df_BindingDB["Library"] = "Protein Binding" 

# Concat the dataframes and convert feature columns to numeric
df_all_2 = pd.concat([df_ROBIN_RNA, df_BindingDB])
df_all_2_columns = df_all_2.columns.tolist()[:-1]
df_all_2[df_all_2_columns] = df_all_2[df_all_2_columns].apply(pd.to_numeric,errors='coerce')

print("Done 1")

# Delete columns that have feature values missing in more than 40% of rows
df_all_2 = df_all_2.dropna(thresh=df_all_2.shape[0]*0.6, how='all',axis=1)

print("Done 2")

# Replace missing and infinity values by median of each column
df_all_2.replace([np.inf, -np.inf], np.nan, inplace=True)
df_all_2 = df_all_2.fillna(df_all_2.median(numeric_only=True))

# Reset the index
df_all_2 = df_all_2.reset_index(drop=True)

df_all_2

In [ ]:
# Save file to the ML_CSVs folder so the preprocessing code does not have to be rerun multiple times
"""
Skip this cell if ROBIN_RNA_vs_BindingDB.csv is already saved in the ML_CSVs folder. 
"""
df_all_2.to_csv(folder_dir + "/ML_CSVs/ROBIN_RNA_vs_BindingDB.csv", 
                index=False)

In [ ]:
"""
If ROBIN_RNA_vs_BindingDB.csv is already saved in the ML_CSVs folder, skip the previous two cells
and read the file ROBIN_RNA_vs_BindingDB.csv here. 
"""
df_all_2 = pd.read_csv(folder_dir + "/ML_CSVs/ROBIN_RNA_vs_BindingDB.csv")
df_all_2

In [ ]:
# Data preparation for ML by conversion to float numpy arrays

# Prepare the features
X_all_2 = df_all_2.values[:, :-1]
X_all_2 = X_all_2.astype('float64')
print("Shape of the X_all_2 matrix:")
print(X_all_2.shape)
print()

# Prepare the labels
y_all_2 = df_all_2.values[:, -1]
y_all_2 = y_all_2.tolist()
y_all_2 = [1 if x == "RNA Binding" else x for x in y_all_2]
y_all_2 = [0 if x == "Protein Binding" else x for x in y_all_2]
y_all_2 = np.array(y_all_2)
y_all_2 = y_all_2.astype(float)
print("Shape of the y_all_2 labels array:")
print(y_all_2.shape)

In [ ]:
# Take 90% of the data to cross validation (stratified) and save the remaining 10% for a final holdout(test) set.

# Import splitting package
from sklearn.model_selection import train_test_split

X_all_2_CV, X_all_2_holdout, y_all_2_CV, y_all_2_holdout = train_test_split(X_all_2, y_all_2,
                                                                            shuffle=True, test_size=0.1, 
                                                                            random_state=10, stratify=y_all_2)

print("Shape of X_all_2 matrix:")
print(X_all_2.shape)
print()

print("Shape of the y_all_2 labels array:")
print(y_all_2.shape)
print()

print("Shape of X_all_2_CV matrix:")
print(X_all_2_CV.shape)
print()

print("Shape of the y_all_2_CV labels array:")
print(y_all_2_CV.shape)
print()

print("Shape of X_all_2_holdout matrix:")
print(X_all_2_holdout.shape)
print()

print("Shape of the y_all_2_holdout labels array:")
print(y_all_2_holdout.shape)

In [ ]:
# Data standardization before the LASSO logistic regression model
# Data used for cross-validation will be standardized together for consistency of the data between cross-validation folds
# The holdout(test) set is standardized using the standardscalar() fit to the X_all_2_CV array.
# The holdout set was not standardized together with the X_all_2_CV array to prevent any data leakage.
# After model selection and optimization using 10-fold cross-validation, the selected model is trained on the X_all_2_CV and tested on the holdout set. 

# Import standardization package
from sklearn.preprocessing import StandardScaler

# Standardize the cross validation features
sc = StandardScaler()
sc.fit(X_all_2_CV)
X_all_2_CV = sc.transform(X_all_2_CV)
print("Shape of the X_all_2_CV features array:")
print(X_all_2_CV.shape)
print()

# Standardize the holdout set features
X_all_2_holdout = sc.transform(X_all_2_holdout)
print("Shape of the X_all_2_holdout features array:")
print(X_all_2_holdout.shape)

In [ ]:
# This cell draws a PR curve with cross validation on the ROBIN RNA binders vs BindingDB classification with LASSO

# Import ML packages
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# Import and reset matplotlib
import matplotlib.pyplot as plt
plt.style.use('default')

# Change font to Helvetica
plt.rc('font', family='Helvetica')

# Import required packages for the PR curve
from sklearn.metrics import average_precision_score, precision_recall_curve

# Set seed value
RANDOM_SEED = 5
np.random.seed(RANDOM_SEED)

# Define the logistic regression model
clf = LogisticRegression(C=0.01, penalty='l1', solver='liblinear',
                         max_iter=100, random_state=RANDOM_SEED, class_weight='balanced')

# Split into 10 stratified folds for cross validation
cv = StratifiedKFold(10, random_state=5, shuffle=True)

# Define the colors to be used for the 10 folds as they are drawn on the PR curve
colors = ["lightblue", "r", "c", "m", "y", "lime", "mediumpurple", "olive", "aqua", "magenta",
          "skyblue", "yellowgreen", "gold", "tan", "silver"]

# Cross validation for loop
LR_y_real = []
LR_y_proba = []
num = 0
for train, test in cv.split(X_all_2_CV, y_all_2_CV):

    # Fit the model on the training set and predict probabilities for the train and test sets
    LR_fit = clf.fit(X_all_2_CV[train], y_all_2_CV[train])
    LR_train_probas = clf.predict_proba(X_all_2_CV[train])
    LR_test_probas = clf.predict_proba(X_all_2_CV[test])

    # Compute PR curve
    precision, recall, _ = precision_recall_curve(y_all_2_CV[test], LR_test_probas[:, 1])

    # Plotting each individual PR Curve and append them to LR_y_real and LR_y_proba lists
    plt.plot(recall, precision, color=colors[num], lw=1, alpha=0.2)
    LR_y_real.append(y_all_2_CV[test])
    LR_y_proba.append(LR_test_probas[:, 1])

    # Update num to keep track of fold number
    num += 1

    # Print Shapes and average precision score for each training and test round
    print("Fold " + str(num) + ":")
    print("Train and Test sets shapes:")
    print(X_all_2_CV[train].shape)
    print(y_all_2_CV[train].shape)
    print(X_all_2_CV[test].shape)
    print(y_all_2_CV[test].shape)
    print("Training set AUPRC:")
    print(round(average_precision_score(y_all_2_CV[train], LR_train_probas[:, 1]), 2))
    print("Test set AUPRC:")
    print(round(average_precision_score(y_all_2_CV[test], LR_test_probas[:, 1]), 2))
    print()

    # Get the LR coefficients and sort them
    LR_coeffs = clf.coef_[0].tolist()

    # Calculate the number of positive and negative logistic regression coefficients
    Num_nonzero_LR_coeffs = 0
    for item in LR_coeffs:
        if item != 0:
            Num_nonzero_LR_coeffs += 1
    print("Num_nonzero_LR_coeffs:")
    print(Num_nonzero_LR_coeffs)
    
    print()
    print("*******************")
    print()

# Plot the no skill line on the bottom of the PR curve
no_skill = len(y_all_2_CV[y_all_2_CV == 1]) / len(y_all_2_CV)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill', color='red')

# Concatenate all LR_y_real and LR_y_proba arrays for drawing of the mean curve
LR_y_real = np.concatenate(LR_y_real)
LR_y_proba = np.concatenate(LR_y_proba)

# Calculate mean precision and recall for drawing the PR curve in the next step
LR_precision, LR_recall, _ = precision_recall_curve(LR_y_real, LR_y_proba)

# Plot the mean PR curve
plt.plot(LR_recall, LR_precision, color='b', 
         label=r'LR: Mean AUPRC (K=10) = %0.2f' % (average_precision_score(LR_y_real, LR_y_proba)),
         lw=2, alpha=1)

# Set aspect ratio
axes = plt.gca()
axes.set_aspect(0.77)

# Remove the gridlines
plt.grid(visible=False)

# Set x-axis and y-axis ticks
tick_list = [0, 0.2, 0.4, 0.6, 0.8, 1]
plt.xticks(tick_list, tick_list, fontsize=15)
plt.yticks(tick_list, tick_list, fontsize=15)
plt.xlabel('Recall', fontsize=20)
plt.ylabel('Precision', fontsize=20)
plt.legend(loc=(0.05, 0.1), fontsize=13)

# Save the PR curve
plt.savefig(folder_dir + '/figures/PRC_ROBIN_RNA_vs_BindingDB.png', dpi=600, bbox_inches="tight")

# Show plot
plt.show()

In [ ]:
# Calculate highest and lowest coefficients of the LASSO logistic regression model

# Train the logistic regression model on the entire dataset this time
clf = LogisticRegression(C=0.01, penalty='l1', solver='liblinear', 
                         max_iter=100, random_state=RANDOM_SEED, class_weight='balanced').fit(X_all_2, y_all_2)

# Get the LR coefficients and sort them
LR_coeffs = clf.coef_[0].tolist()
sorted_LR_coeffs = sorted(LR_coeffs, reverse=True)

# Calculate the number of positive and negative logistic regression coefficients
Num_positive_LR_coeffs = 0
for item in LR_coeffs:
    if item > 0:
        Num_positive_LR_coeffs += 1
Num_negative_LR_coeffs = 0
for item in LR_coeffs:
    if item < 0:
        Num_negative_LR_coeffs += 1

# Get the positive and negative coefficients as lists
sorted_LR_coeffs_top = sorted_LR_coeffs[0:Num_positive_LR_coeffs]
sorted_LR_coeffs_buttom = sorted_LR_coeffs[-1*Num_negative_LR_coeffs:][::-1]

# Get features of the df_all_3 matrix
features = df_all_2.columns.tolist()[1:-1]

# Get the names of features with the highest positive coefficients in the LR model
top_features = []
for j in range(len(sorted_LR_coeffs_top)):
    for i in range(len(LR_coeffs)):
        if sorted_LR_coeffs_top[j] == LR_coeffs[i]:
            top_features.append((features[i], round(LR_coeffs[i], 3)))
print("Positive coefficients and their features:")
print(top_features)
print()

# Get the names of features with the lowest negative coefficients in the LR model
buttom_features = []
for j in range(len(sorted_LR_coeffs_buttom)):
    for i in range(len(LR_coeffs)):
        if sorted_LR_coeffs_buttom[j] == LR_coeffs[i]:
            buttom_features.append((features[i], round(LR_coeffs[i], 3)))
print("Negative coefficients and their features:")
print(buttom_features)


# Report the number of non-zero features in the LR model
count = 0
for item in sorted_LR_coeffs:
    if item > 0 or item < 0:
        count += 1
print()
print("Number of nonzero coefficients:")
print(count)

# Save the non-zero features and their coefficients in the supplemental folder
df_LASSO_nonzero_features = pd.DataFrame(top_features+buttom_features, columns=['Feature', 'LR Coefficient'])
df_LASSO_nonzero_features.to_csv(folder_dir +
                                 '/LASSO_coefficients/LASSO_all_nonzero_features_ROBIN_RNA_vs_BindingDB.csv',
                                 index=False)

In [ ]:
# Train the model on the entire CV set and test on the holdout set define earlier with relevant performance metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score

# Define the model
clf = LogisticRegression(C=0.01, penalty='l1', solver='liblinear',
                         max_iter=100, random_state=RANDOM_SEED, class_weight='balanced')

# Fit the model on the training set and predict probabilities for the train and test sets
LR_fit = clf.fit(X_all_2_CV, y_all_2_CV)
prediction_probs = clf.predict_proba(X_all_2_holdout)[:, 1]
prediction_classes = clf.predict(X_all_2_holdout)

cm = confusion_matrix(y_all_2_holdout, prediction_classes)
print("Confusion matrix on the holdout set:")
print(cm)
print()

AUPRC_score_holdout = average_precision_score(y_all_2_holdout, prediction_probs)
print("AUPRC score on the holdout set:")
print(round(AUPRC_score_holdout, 2))
print()

f1_score_holdout = f1_score(y_all_2_holdout, prediction_classes)
print("F1 score on the holdout set:")
print(round(f1_score_holdout, 2))
print()

AUROC_score_holdout = roc_auc_score(y_all_2_holdout, prediction_probs)
print("AUROC score on the holdout set:")
print(round(AUROC_score_holdout, 2))
print()

cohen_kappa_score_holdout = cohen_kappa_score(y_all_2_holdout, prediction_classes)
print("Cohen’s kappa score on the holdout set:")
print(round(cohen_kappa_score_holdout, 2))
print()

matthews_corrcoef_holdout = matthews_corrcoef(y_all_2_holdout, prediction_classes)
print("Matthews correlation coefficient on the holdout set:")
print(round(matthews_corrcoef_holdout, 2))
print()

## b) Classification of augmented ROBIN RNA binders and BindingDB protein binders

In [ ]:
# In this part of the code, we classify augmented ROBIN RNA and BindingDB protein binders.

# Read in the ROBIN RNA binders, ROBIN augmented library, and BindingDB protein binders
df_ROBIN_RNA = pd.read_csv(folder_dir + "/data/Mordred_files/Mordred_ROBIN_RNA_Binder_3D.csv")
df_Augmented_ROBIN = pd.read_csv(folder_dir + "/data/Mordred_files/Mordred_Augmented_ROBIN_RNA_3D.csv")
df_BindingDB = pd.read_csv(folder_dir + "/data/Mordred_files/Mordred_BindingDB_3D.csv")

# ______________________________________________

# Report the number of compounds in each library that will be used in machine learning
print("Number of compounds in the BindingDB protein binding library:")
print(df_BindingDB.shape[0])
print()

print("Number of compounds in the Augmented_ROBIN library:")
print(df_Augmented_ROBIN.shape[0])
print()

print("Number of compounds in the ROBIN RNA Binding library:")
print(df_ROBIN_RNA.shape[0])
print()

### Machine Learning Strategy:

Two matrices wil be utilized to train the augmented ROBIN RNA binders vs BindingDB classifiers.

Matrix 1 (df_all_3): This matrix is comprised of augmented ROBIN RNA binders and BindingDB protein binders. This is the matrix whose standardized chemical descriptor values will be used throughout for training and testing the dataset. 

Matrix 2 (df_all_4): This matrix only has experimentally-derived ROBIN RNA binders and BindingDB protein binders and their features. This matrix does not have any computational augmentation at any stage. This matrix is used for splitting the dataset and defining the train and test sets in each round of cross-validation. Only compound names of this matrix are used in the machine learning steps. 

### 10-fold Cross-Validation Strategy Step by Step:

1. Matrix df_all_4 (matrix 2) is split in a stratified manner (same class balance as the original dataset) into 10 folds. Splitting of the matrix that is not augmented ensures only experimenal compounds end up in each test set. 
2. In each round of cross validation, one of the 10 folds is assigned as a test set.
3. Compounds in each of the test sets are first extracted. Then, BindingDB compounds of the test set are taken out of df_all_3. For ROBIN RNA binders in each test set, the compounds themselves and all their analogues are taken out of df_all_3 to prevent any data leakage from train to test set.
4. The new df_all_3 with the test set compounds and their analogues taken out is trained on with the machine learning models. Models are tested on the test set compounds extracted from the experimental df_all_4 matrix. All features are standardized initially in the df_all_3 matrix so rows of compound features are exclusively extracted from the df_all_3 matrix.

** Note: As a result of this cross-validation strategy, the train and test sets have different class balances; therefore, higher performance of the model on the training set might not be indicative of overfitting.


In [ ]:
# Combine the two dataframes and conduct further processing

"""
This cell might take a long time to run. This code does not have to be rerun if 
Augmented_ROBIN_RNA_vs_BindingDB.csv is already saved in the ML_CSVs folder. 
"""

# Assigning a library name to the two dataframes
df_BindingDB["library"] = "Protein Binding"
df_Augmented_ROBIN["library"] = "RNA Binding"

# Concat the two dataframes
df_all_3 = pd.concat([df_BindingDB, df_Augmented_ROBIN])

# Get the list of all feature columns in the dataset and coerce them to numeric
df_all_3_columns = df_all_3.columns.tolist()[1:-1]
df_all_3[df_all_3_columns] = df_all_3[df_all_3_columns].apply(pd.to_numeric, errors='coerce')

print("Done1")
# Delete columns that have feature values missing in more than 60% of compounds
df_all_3 = df_all_3.dropna(thresh=df_all_3.shape[0]*0.6, how='all', axis=1)

print("Done2")
# Replace missing and infinity values by median of each column
df_all_3.replace([np.inf, -np.inf], np.nan, inplace=True)
df_all_3 = df_all_3.fillna(df_all_3.median(numeric_only=True))

# Reset the index
df_all_3 = df_all_3.reset_index(drop=True)
df_all_3

In [ ]:
# Save the preprocessed dataframe so the data does not have to reprocessed with every notebook
"""
Skip this cell if Augmented_ROBIN_RNA_vs_BindingDB.csv is already saved in the ML_CSVs folder. 
"""
df_all_3.to_csv(folder_dir + "/ML_CSVs/Augmented_ROBIN_RNA_vs_BindingDB.csv", index=False)

In [ ]:
# Load back the preprocessed dataframe
"""
If Augmented_ROBIN_RNA_vs_BindingDB.csv is already saved in the ML_CSVs folder, skip the previous two cells
and read the file Augmented_ROBIN_RNA_vs_BindingDB.csv here.
"""
df_all_3 = pd.read_csv(folder_dir + "/ML_CSVs/Augmented_ROBIN_RNA_vs_BindingDB.csv")
df_all_3

In [ ]:
# Data preparation and standardization before the LASSO logistic regression and neural network models
# df_all_3 is the augmented dataset combined
from sklearn.preprocessing import StandardScaler

# Prepare the features
X_all_3 = df_all_3.values[:, 1:-1]
X_all_3 = X_all_3.astype('float64')
sc = StandardScaler()
sc.fit(X_all_3)
X_all_3 = sc.transform(X_all_3)
print("Shape of X_all_3 matrix:")
print(X_all_3.shape)
print()

# Prepare the labels
y_all_3 = df_all_3.values[:, -1]
y_all_3 = y_all_3.tolist()
y_all_3 = [1 if x == "RNA Binding" else x for x in y_all_3]
y_all_3 = [0 if x == "Protein Binding" else x for x in y_all_3]
y_all_3 = np.array(y_all_3)
y_all_3 = y_all_3.astype(float)
print("Shape of the y_all_3 labels array:")
print(y_all_3.shape)

In [ ]:
# Convert standardized df_all_3 back to array since Pandas operations will be used on this dataframe later
# Also, add the name and label columns back in as they will be used later
df_all_3_feature_columns = df_all_3.columns.tolist()[1:-1]
df_all_3_name_label_standardized = pd.DataFrame(X_all_3, columns=df_all_3_feature_columns)
df_all_3_compound_names = pd.Series(df_all_3["name"].tolist(), name="name")
df_all_3_name_label_standardized = pd.concat([df_all_3_compound_names, df_all_3_name_label_standardized], axis=1)
df_all_3_labels = pd.Series(y_all_3.tolist(), name="label")
df_all_3_name_label_standardized = pd.concat([df_all_3_name_label_standardized, df_all_3_labels], axis=1)
df_all_3_name_label_standardized

In [ ]:
# Preproccess df_all_4 and save so the preprocessing does not have to be rerun
# df_all_4 is the non-augmented dataset which will be used for testing the performance of the augmented set in each fold of cross validation

"""
This cell might take a long time to run. This code does not have to be rerun if 
ROBIN_RNA_vs_BindingDB_used_in_Augmentation.csv is already saved in the ML_CSVs folder. 
"""

# Add libary names to the dataframes
df_ROBIN_RNA["library"] = "RNA Binding"
df_BindingDB["library"] = "Protein Binding" 


df_all_4 = pd.concat([df_ROBIN_RNA, df_BindingDB])

df_all_3_features = df_all_3.columns.tolist()
df_all_4 = df_all_4[df_all_3_features]

df_all_4_features = df_all_4.columns.tolist()[1:-1]
df_all_4[df_all_4_features] = df_all_4[df_all_4_features].apply(pd.to_numeric, errors='coerce')

print("Done1")


# Replace missing and infinity values by median of each column
df_all_4.replace([np.inf, -np.inf], np.nan, inplace=True)
df_all_4 = df_all_4.fillna(df_all_4.median(numeric_only=True))


print("Done2")

# Reset the index
df_all_4 = df_all_4.reset_index(drop=True)

df_all_4.to_csv(folder_dir + "/ML_CSVs/ROBIN_RNA_vs_BindingDB_used_in_Augmentation.csv", index=False)

df_all_4

In [ ]:
"""
If ROBIN_RNA_vs_BindingDB_used_in_Augmentation.csv is already saved in the ML_CSVs folder, skip the previous cell
and read the file Augmented_ROBIN_RNA_vs_BindingDB.csv here. 
"""

# Load back df_all_4 and standardize
df_all_4 = pd.read_csv(folder_dir + "/ML_CSVs/ROBIN_RNA_vs_BindingDB_used_in_Augmentation.csv")
#prepare the features
X_all_4 = df_all_4.values[:, :-1]
print("Shape of X_all_4 matrix:")
print(X_all_4.shape)
print()

#prepare the labels
y_all_4 = df_all_4.values[:,-1]
y_all_4 = y_all_4.tolist()
y_all_4 = [1 if x=="RNA Binding" else x for x in y_all_4]
y_all_4 = [0 if x=="Protein Binding" else x for x in y_all_4]
y_all_4 = np.array(y_all_4)
y_all_4 = y_all_4.astype(float)
print("Shape of the y_all_4 labels array:")
print(y_all_4.shape)


In [ ]:
# Take 90% of the data to cross validation (stratified) and save the remaining 10% for a final holdout(test) set. 

from sklearn.model_selection import train_test_split

X_all_4_CV, X_all_4_holdout, y_all_4_CV, y_all_4_holdout = train_test_split(X_all_4, y_all_4,shuffle = True, test_size=0.1, random_state=10, stratify = y_all_4)

print ("Shape of X_all_4 matrix:")
print (X_all_4.shape)
print ()

print ("Shape of the y_all_4 labels array:")
print (y_all_4.shape)
print ()

print ("Shape of X_all_4_CV matrix:")
print (X_all_4_CV.shape)
print ()

print ("Shape of the y_all_4_CV labels array:")
print (y_all_4_CV.shape)
print ()

print ("Shape of X_all_4_holdout matrix:")
print (X_all_4_holdout.shape)
print ()

print ("Shape of the y_all_4_holdout labels array:")
print (y_all_4_holdout.shape)



In [ ]:
# Load in the dictionary that maps each ROBIN RNA binder to 30 analogues for augmentation 
# This dictionary will be used to prevent data leakage during cross-validation to exclude analogues of test cases in the train set
df_ROBIN_Augment_map = pd.read_csv(folder_dir + "/ML_CSVs/ROBIN_Plus_Dictionary.csv")
df_ROBIN_Augment_map_names_og = df_ROBIN_Augment_map["Name"].tolist()
df_ROBIN_Augment_map_names = df_ROBIN_Augment_map["ROBIN_Plus_Name"].tolist()
df_ROBIN_Augment_map_names_augmented = []
for item in df_ROBIN_Augment_map_names:
    df_ROBIN_Augment_map_names_augmented.append(item.split(";"))
ROBIN_Augment_dict = {}
for i in range(len(df_ROBIN_Augment_map_names_og)):
    ROBIN_Augment_dict[df_ROBIN_Augment_map_names_og[i]] =  df_ROBIN_Augment_map_names_augmented[i]
ROBIN_Augment_dict 

In [ ]:
# Remove warnings
import warnings
warnings.filterwarnings('ignore')

## Next step removes the holdout set compounds from the set used for cross validation

In [ ]:
# Exclude holdout(test) set compounds and their analogues from the set of compounds used for cross-validation
# This operation will be performed on the df_all_3_name_label_standardized dataframe.

# RNA compounds to exclude from the CV set
test_set_compounds = X_all_4_holdout[:, 0].tolist()
test_set_RNA_compounds = [item for item in test_set_compounds if item[0:9] != "BindingDB"]
test_set_RNA_compounds_expanded = []
for item in test_set_RNA_compounds:
    test_set_RNA_compounds_expanded.append(item)
    item_expanded = ROBIN_Augment_dict[item]
    for thing in item_expanded:
        test_set_RNA_compounds_expanded.append(thing)
test_set_RNA_compounds_expanded_set = list(set(test_set_RNA_compounds_expanded))

# BindingDB compounds to exclude from the CV set
test_set_BindingDB = [item for item in test_set_compounds if item[0:9] == "BindingDB"]

# BindingDB and RNA binders to be excluded from the CV set combined into one set
test_set_RNA_expanded_and_BindingDB = test_set_BindingDB + test_set_RNA_compounds_expanded_set

# Remove from df_all_3_name_label_standardized BindingDB compounds from the test set
# Also remove from df_all_3_name_label_standardized test set ROBIN RNA binders and their analogues
df_all_3_name_label_standardized_CV = df_all_3_name_label_standardized.loc[~df_all_3_name_label_standardized['name'].isin(test_set_RNA_expanded_and_BindingDB)]
# df_all_3_name_label_standardized_CV is the cross-validation set

# Print shape of the CV set and the number of protein and RNA binders within it
print("Shape of the augmented cross-validation set:")
print(df_all_3_name_label_standardized_CV.shape)
y_all_3_CV_labels_list = df_all_3_name_label_standardized_CV.values[:, -1].tolist()
y_all_3_CV_labels_list = [1 if x == "RNA Binding" else x for x in y_all_3_CV_labels_list]
y_all_3_CV_labels_list = [0 if x == "Protein Binding" else x for x in y_all_3_CV_labels_list]
print("Number of protein binders:")
print(int(len(y_all_3_CV_labels_list) - sum(y_all_3_CV_labels_list)))
print("Number of RNA binders:")
print(int(sum(y_all_3_CV_labels_list)))
print()

## Next step does the splitting as described earlier and saves each train and test set for each fold in the ML_CSVs folder. This step will ensure the splitting does not have be to be redone after the initial run of the next cell. 

In [ ]:
# Save the folds used for cross validation as CSVs for faster execution when code is run again

"""
This cell might take a long time to run. This code does not have to be rerun if 
CSVs of all folds are already saved in the Augmented_ROBIN_vs_BindingDB_Stratified_Folds folder within the 
ML_CSVs folder. 
"""


# Split into 10 folds in a stratified manner 
cv = StratifiedKFold(10, random_state = 5, shuffle = True)

num = 1
for train, test in cv.split(X_all_4_CV, y_all_4_CV):
    
    # Print fold number 
    print ("Fold " + str(num) + ":")

    # RNA compounds to exclude from the training set
    test_set_compounds = X_all_4_CV[test][:, 0].tolist()
    test_set_RNA_compounds = [item for item in test_set_compounds if item[0:9] != "BindingDB"]
    test_set_RNA_compounds_expanded = []
    for item in test_set_RNA_compounds:
        test_set_RNA_compounds_expanded.append(item)
        item_expanded = ROBIN_Augment_dict[item]
        for thing in item_expanded:
            test_set_RNA_compounds_expanded.append(thing)
    test_set_RNA_compounds_expanded_set = list(set(test_set_RNA_compounds_expanded))

    # BindingDB compounds to exclude from the training set
    test_set_BindingDB = [item for item in test_set_compounds if item[0:9] == "BindingDB"]

    # BindingDB and RNA binders to be excluded from the train set combined into one set
    test_set_RNA_expanded_and_BindingDB = test_set_BindingDB + test_set_RNA_compounds_expanded_set
    
    # Remove from df_all_3_name_label_standardized_CV BindingDB compounds from the test set
    # Also remove from df_all_3_name_label_standardized_CV test set ROBIN RNA binders and their analogues
    df_all_3_name_label_standardized_train = df_all_3_name_label_standardized_CV.loc[~df_all_3_name_label_standardized_CV['name'].isin(test_set_RNA_expanded_and_BindingDB)]
    # df_all_3_name_label_standardized_train is the train set
    
    # Print shape of the train set and the number of protein and RNA binders within it
    print("Shape of Augmented Train set:")
    print (df_all_3_name_label_standardized_train.shape)
    df_all_3_name_label_standardized_train_labels = df_all_3_name_label_standardized_train["label"].tolist()
    print("Number of protein binders:")
    print(int(len(df_all_3_name_label_standardized_train_labels) - sum(df_all_3_name_label_standardized_train_labels)))
    print("Number of RNA binders:")
    print(int(sum(df_all_3_name_label_standardized_train_labels)))
    print()

    # Extract from df_all_3_name_label_standardized_CV the test set compounds belonging to BindingDB and ROBIN RNA binders
    test_set_RNA_and_BindingDB = test_set_RNA_compounds + test_set_BindingDB
    df_all_3_name_label_standardized_test = df_all_3_name_label_standardized_CV.loc[df_all_3_name_label_standardized_CV['name'].isin(test_set_RNA_and_BindingDB)]
    df_all_3_name_label_standardized_test.drop_duplicates(subset=['name'], keep="first", inplace=True)
    # df_all_3_name_label_standardized_test is the test set
    # Test set compounds are extracted again from df_all_3_name_label_standardized_CV to ensure standardization of train and test sets are the same
    
    # Print shape of the test set and the number of protein and RNA binders within it
    print("Shape of test set:")
    print(df_all_3_name_label_standardized_test.shape)
    df_all_3_name_label_standardized_test_labels = df_all_3_name_label_standardized_test["label"].tolist()
    print("Number of protein binders:")
    print(int(len(df_all_3_name_label_standardized_test_labels) - sum(df_all_3_name_label_standardized_test_labels)))
    print("Number of RNA binders:")
    print(int(sum(df_all_3_name_label_standardized_test_labels)))

    # Get the feature values and labels from the pandas dataframes
    X_3_all_Augmented_train = df_all_3_name_label_standardized_train.iloc[:, 1:-1]
    y_3_all_Augmented_train = df_all_3_name_label_standardized_train.iloc[:, -1]

    X_3_all_Augmented_test = df_all_3_name_label_standardized_test.iloc[:, 1:-1]
    y_3_all_Augmented_test = df_all_3_name_label_standardized_test.iloc[:, -1]
    
    # Save in Augmented_ROBIN_vs_BindingDB_Stratified_Folds folder within the ML_CSVs folder
    save_dir = folder_dir + "/ML_CSVs/Augmented_ROBIN_vs_BindingDB_Stratified_Folds/" + "fold_" + str(num) + "/"
    
    # Save the files in their corresponding CSVs
    X_3_all_Augmented_train.to_csv(save_dir + "X_train" + ".csv", index=False)
    y_3_all_Augmented_train.to_csv(save_dir + "y_train" + ".csv", index=False)
    X_3_all_Augmented_test.to_csv(save_dir + "X_test" + ".csv", index=False)
    y_3_all_Augmented_test.to_csv(save_dir + "y_test" + ".csv", index=False)
    
    #Update num
    num += 1
    
    print ()
    print("*****************")
    print ()

In [ ]:
"""
If folds are already saved in the Augmented_ROBIN_vs_BindingDB_Stratified_Folds folder within the ML_CSVs folder, 
skip the previous cell and read each fold's train and test sets here. 
"""

# Read the saved folds back in 

for k in range(1,11):
    
    print (k)
    
    exec(f'fold_{k}_X_train = pd.read_csv(folder_dir + "/ML_CSVs/Augmented_ROBIN_vs_BindingDB_Stratified_Folds/" + "fold_" + str(k) + "/" + "X_train.csv")')
  
    exec(f'fold_{k}_y_train = pd.read_csv(folder_dir + "/ML_CSVs/Augmented_ROBIN_vs_BindingDB_Stratified_Folds/" + "fold_" + str(k) + "/" + "y_train.csv")')

    exec(f'fold_{k}_X_test = pd.read_csv(folder_dir + "/ML_CSVs/Augmented_ROBIN_vs_BindingDB_Stratified_Folds/" + "fold_" + str(k) + "/" + "X_test.csv")')

    exec(f'fold_{k}_y_test = pd.read_csv(folder_dir + "/ML_CSVs/Augmented_ROBIN_vs_BindingDB_Stratified_Folds/" + "fold_" + str(k) + "/" + "y_test.csv")')

# Example file read from fold_1_X_train
fold_1_X_train

## LASSO Logistic Regression Model:

In [ ]:
# Perform cross-validation on LASSO logistic regression model

# Import required packages for the PR curve
from sklearn.metrics import average_precision_score, precision_recall_curve, confusion_matrix
from sklearn.metrics import f1_score

# Define the logistic regression model
clf = LogisticRegression(C=0.005, penalty='l1', solver='liblinear',
                         max_iter=100, random_state=RANDOM_SEED, class_weight='balanced')

# Define the colors to be used for the 10 folds as they are drawn on the PR curve
colors = ["lightblue", "r", "c", "m", "y", "lime", "mediumpurple", "olive", "aqua",
          "magenta", "skyblue", "yellowgreen", "gold", "tan", "silver"]

# Cross-validation for loop
LR_y_real = []
LR_y_proba = []
num = 0
for k in range(1, 11):

    # Print fold number
    print("Fold " + str(num+1) + ":")

    # Read in the fetures and labels for the train and test sets read earlier
    X_train = vars()['fold_' + str(k) + "_X_train"]
    y_train = vars()['fold_' + str(k) + "_y_train"]
    X_test = vars()['fold_' + str(k) + "_X_test"]
    y_test = vars()['fold_' + str(k) + "_y_test"]

    # Update num
    num += 1

    # Convert the dataframes to numpy arrays for machine learning
    X_3_all_Augmented_train = X_train.values
    y_3_all_Augmented_train = y_train.values
    X_3_all_Augmented_test = X_test.values
    y_3_all_Augmented_test = y_test.values

    # Fit the model on the training set and predict probabilities for the train and test sets
    LR_fit = clf.fit(X_3_all_Augmented_train, y_3_all_Augmented_train)
    LR_test_probas = clf.predict_proba(X_3_all_Augmented_test)
    LR_train_probas = clf.predict_proba(X_3_all_Augmented_train)


    # Compute PR and area under the curve
    precision, recall, _ = precision_recall_curve(y_3_all_Augmented_test, LR_test_probas[:, 1])

    # Plotting each individual PR Curve and append them to LR_y_real and LR_y_proba lists
    plt.plot(recall, precision, color=colors[num], lw=1, alpha=0.2)
    LR_y_real.append(y_3_all_Augmented_test)
    LR_y_proba.append(LR_test_probas[:, 1])

    # Get model class predictions instead of probabilities to show the confusion matrices
    prediction_classes_test = clf.predict(X_3_all_Augmented_test)
    prediction_classes_train = clf.predict(X_3_all_Augmented_train)

    # Draw confusion matrix of the model performance on the train set
    cm = confusion_matrix(y_3_all_Augmented_train, prediction_classes_train)
    print("Confusion matrix on the test set:")
    print(cm)

    # Draw confusion matrix of the model performance on the test set
    cm = confusion_matrix(y_3_all_Augmented_test, prediction_classes_test)
    print("Confusion matrix on the test set:")
    print(cm)

    # Print F1 score and average prevision score for train and test sets in each round
    print("F1 score test:")
    print(round(f1_score(y_3_all_Augmented_test, prediction_classes_test), 2))
    print("F1 score train:")
    print(round(f1_score(y_3_all_Augmented_train, prediction_classes_train), 2))
    print()
    print("Test AVG precision score:")
    print(round(average_precision_score(y_3_all_Augmented_test, LR_test_probas[:, 1]),2))
    print("Train AVG precision score:")
    print(round(average_precision_score(y_3_all_Augmented_train, LR_train_probas[:, 1]),2))
    print()

    # Get the LR coefficients and sort them
    LR_coeffs = clf.coef_[0].tolist()

    # Calculate the number of positive and negative logistic regression coefficients
    Num_nonzero_LR_coeffs = 0
    for item in LR_coeffs:
        if item != 0:
            Num_nonzero_LR_coeffs += 1

    # Report number of nonzero LR coefficients
    print("Num_nonzero_LR_coeffs:") 
    print(Num_nonzero_LR_coeffs)

    print()
    print("*******************")
    print()

# Plot the no skill line on the bottom of the PR curve
no_skill = len(y_all_2_CV[y_all_2_CV == 1]) / len(y_all_2_CV)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill', color='red')

# Concatenate all LR_y_real and LR_y_proba arrays for drawing of the mean curve
LR_y_real = np.concatenate(LR_y_real)
LR_y_proba = np.concatenate(LR_y_proba)

# Calculate mean precision and recall for drawing the PR curve in the next step
LR_precision, LR_recall, thresholds = precision_recall_curve(LR_y_real, LR_y_proba)

# Plot the mean PR curve
plt.plot(LR_recall, LR_precision, color='b',
             label=r'LR: Mean AUPRC (K=10) = %0.2f' % (average_precision_score(LR_y_real, LR_y_proba)),
             lw=2, alpha=1)

# Set aspect ratio
axes = plt.gca()
axes.set_aspect(0.75)

# Remove the gridlines
plt.grid(visible=False)


# Set x-axis and y-axis ticks
tick_list = [0, 0.2, 0.4, 0.6, 0.8, 1]
plt.xticks(tick_list, tick_list, fontsize=15)
plt.yticks(tick_list, tick_list, fontsize=15)
plt.xlabel('Recall', fontsize=20)
plt.ylabel('Precision', fontsize=20)
plt.legend(loc="lower left", fontsize=13)


# save the PR curve
plt.savefig(folder_dir + '/figures/PRC_Augmented_ROBIN_RNA_vs_BindingDB_LR.png',
            dpi=600, bbox_inches="tight")

# Show plot
plt.show()

In [ ]:
# Calculate highest and lowest coefficients of the LASSO logistic regression model

# Train the logistic regression model on the entire dataset this time
clf = LogisticRegression(C=0.005, penalty='l1', solver='liblinear', 
                         max_iter=100, random_state=RANDOM_SEED, class_weight='balanced').fit(X_all_3, y_all_3)

# Get the LR coefficients and sort them
LR_coeffs = clf.coef_[0].tolist()
sorted_LR_coeffs = sorted(LR_coeffs, reverse=True)

# Calculate the number of positive and negative logistic regression coefficients
Num_positive_LR_coeffs = 0
for item in LR_coeffs:
    if item > 0:
        Num_positive_LR_coeffs += 1
Num_negative_LR_coeffs = 0
for item in LR_coeffs:
    if item < 0:
        Num_negative_LR_coeffs += 1

# Get the positive and negative coefficients as lists
sorted_LR_coeffs_top = sorted_LR_coeffs[0:Num_positive_LR_coeffs]
sorted_LR_coeffs_buttom = sorted_LR_coeffs[-1*Num_negative_LR_coeffs:][::-1]

# Get features of the df_all_3 matrix
features = df_all_3.columns.tolist()[1:-1]

# Get the names of features with the highest positive coefficients in the LR model
top_features = []
for j in range(len(sorted_LR_coeffs_top)):
    for i in range(len(LR_coeffs)):
        if sorted_LR_coeffs_top[j] == LR_coeffs[i]:
            top_features.append((features[i], round(LR_coeffs[i], 3)))
print("Positive coefficients and their features:")
print(top_features)
print()

# Get the names of features with the lowest negative coefficients in the LR model
buttom_features = []
for j in range(len(sorted_LR_coeffs_buttom)):
    for i in range(len(LR_coeffs)):
        if sorted_LR_coeffs_buttom[j] == LR_coeffs[i]:
            buttom_features.append((features[i], round(LR_coeffs[i], 3)))
print("Negative coefficients and their features:")
print(buttom_features)


# Report the number of non-zero features in the LR model
count = 0
for item in sorted_LR_coeffs:
    if item > 0 or item < 0:
        count += 1
print()
print("Number of nonzero coefficients:")
print(count)

# Save the non-zero features and their coefficients in the supplemental folder
df_LASSO_nonzero_features = pd.DataFrame(top_features+buttom_features, columns=['Feature', 'LR Coefficient'])
df_LASSO_nonzero_features.to_csv(folder_dir +
                                 '/LASSO_coefficients/LASSO_all_nonzero_features_Augmented_ROBIN_RNA_vs_BindingDB.csv',
                                 index=False)

# MLP Neural Network Model:

In [ ]:
# Import the neural network packages
import tensorflow as tf
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import Sequential
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# Define the neural network MLP model and wrap it in the scikitlearn pipeline for easier use

# Define neural network as function
def create_model():

    # create model
    model = Sequential()

    # Define number of features
    number_of_features = 1665

    # Hidden layers
    model.add(layers.Dense(units=200, activation='relu', kernel_regularizer=regularizers.l2(0.00001),
                           input_shape=(number_of_features,)))

    model.add(layers.Dense(units=200, activation='relu', kernel_regularizer=regularizers.l2(0.00001)))

    # Output layer
    model.add(layers.Dense(units=1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy',  # Cross-entropy
                  optimizer='Adam',  # Use Adam optimizer
                  metrics=['AUC'])
    return model


# Wrap within the scikitlearn framework
MLP_model = KerasClassifier(create_model, epochs=7, batch_size=2048, verbose=1)

In [ ]:
# Use the MLP neural network model with the same cross-validation strategy

# Import required packages for the PR curve
from sklearn.metrics import average_precision_score, precision_recall_curve, confusion_matrix
from sklearn.metrics import f1_score

# Import the package for adjusting class weights for the neural net
from sklearn.utils import compute_class_weight

# Set plot font to Halvetica
plt.rc('font', family='Helvetica')

# Set random seed for numpy and tensorflow
RANDOM_SEED = 5
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Make sure matplotlib settings are reset
plt.style.use('default')

# Define the colors to be used for the 10 folds as they are drawn on the PR curve
colors = ["lightblue", "r", "c", "m", "y", "lime", "mediumpurple", "olive", "aqua",
          "magenta", "skyblue", "yellowgreen", "gold", "tan", "silver"]

# Cross-validation for loop
MLP_y_real = []
MLP_y_proba = []
num = 0
for k in range(1, 11):

    # Print fold number
    print("Fold " + str(num+1) + ":")

    # Read in the fetures and labels for the train and test sets read earlier
    X_train = vars()['fold_' + str(k) + "_X_train"]
    y_train = vars()['fold_' + str(k) + "_y_train"]
    X_test = vars()['fold_' + str(k) + "_X_test"]
    y_test = vars()['fold_' + str(k) + "_y_test"]

    # Update num
    num += 1

    # Convert the dataframes to numpy arrays for machine learning
    X_3_all_Augmented_train = X_train.values
    y_3_all_Augmented_train = y_train.values
    X_3_all_Augmented_test = X_test.values
    y_3_all_Augmented_test = y_test.values

    # There is still a slight imbalance in the train set so adjust the weights accordingly
    y_3_all_Augmented_train_list = [x[0] for x in y_3_all_Augmented_train.tolist()]
    class_weights = compute_class_weight(class_weight="balanced",
                                        classes=np.unique(y_3_all_Augmented_train_list),
                                        y=y_3_all_Augmented_train_list)
    class_weights = dict(zip(np.unique(y_3_all_Augmented_train_list), class_weights))
    print("Class weights:")
    print(class_weights)
    print()

    # Fit the model on the training set and predict probabilities for the train and test sets
    MLP_fit = MLP_model.fit(X_3_all_Augmented_train, y_3_all_Augmented_train, class_weight=class_weights)
    MLP_test_probas = MLP_model.predict_proba(X_3_all_Augmented_test)
    MLP_train_probas = MLP_model.predict_proba(X_3_all_Augmented_train)

    # Compute PR and area under the curve
    precision, recall, _ = precision_recall_curve(y_3_all_Augmented_test, MLP_test_probas[:, 1])

    # Plot each individual PR Curve and append to MLP_y_real and MLP_y_proba lists
    plt.plot(recall, precision, color=colors[num], lw=1, alpha=0.2)
    MLP_y_real.append(y_3_all_Augmented_test)
    MLP_y_proba.append(MLP_test_probas[:, 1])

    # Get model class predictions instead of probabilities to show the confusion matrices
    prediction_classes_test = MLP_model.predict(X_3_all_Augmented_test)
    prediction_classes_train = MLP_model.predict(X_3_all_Augmented_train)

    # Draw confusion matrix of the model performance on the train set
    cm = confusion_matrix(y_3_all_Augmented_train, prediction_classes_train)
    print("Confusion matrix on the test set:")
    print(cm)

    # Draw confusion matrix of the model performance on the test set
    cm = confusion_matrix(y_3_all_Augmented_test, prediction_classes_test)
    print("Confusion matrix on the test set:")
    print(cm)

    # Print F1 score and average prevision score for train and test sets in each round
    print("F1 score test:")
    print(round(f1_score(y_3_all_Augmented_test, prediction_classes_test), 2))
    print("F1 score train:")
    print(round(f1_score(y_3_all_Augmented_train, prediction_classes_train), 2))
    print()
    print("Test AVG precision score:")
    print(round(average_precision_score(y_3_all_Augmented_test, MLP_test_probas[:, 1]), 2))
    print("Train AVG precision score:")
    print(round(average_precision_score(y_3_all_Augmented_train, MLP_train_probas[:, 1]), 2))
    print()
    print("*******************")
    print()

# Plot the no skill line on the bottom of the PR curve
no_skill = len(y_all_4_CV[y_all_4_CV == 1]) / len(y_all_4_CV)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill', color='r')

# Concatenate all MLP_y_real and MLP_y_proba arrays for drawing of the mean curve
MLP_y_real = np.concatenate(MLP_y_real)
MLP_y_proba = np.concatenate(MLP_y_proba)

# Calculate mean precision and recall for drawing the PR curve in the next step
MLP_precision, MLP_recall, thresholds = precision_recall_curve(MLP_y_real, MLP_y_proba)

# Plot the mean PR curve
plt.plot(MLP_recall, MLP_precision, color='b',
             label=r'MLP: Mean AUPRC (K=10) = %0.2f' % (average_precision_score(MLP_y_real, MLP_y_proba)),
             lw=2, alpha=1)

# Set aspect ratio
axes = plt.gca()
axes.set_aspect(0.75)

# Remove the gridlines
plt.grid(visible=False)


# Set x-axis and y-axis ticks
tick_list = [0, 0.2, 0.4, 0.6, 0.8, 1]
plt.xticks(tick_list, tick_list, fontsize=15)
plt.yticks(tick_list, tick_list, fontsize=15)
plt.xlabel('Recall', fontsize=20)
plt.ylabel('Precision', fontsize=20)
plt.legend(loc="lower left", fontsize=13)


# Save the PR curve
plt.savefig(folder_dir + '/figures/PRC_Augmented_ROBIN_RNA_vs_BindingDB_MLP.png', dpi=600, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Test the performance of the MLP model on the holdout/test set and report relevant metrics

# Import packages required for calculation of different metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

# Set random seed for numpy and tensorflow
RANDOM_SEED = 5
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Print initial shape of df_all_3 for reference
print("Shape of df_all_3:")
print(df_all_3.shape)
print()

'''
The same strategy as cross validation is used here but instead of a cross validation test set, X_all_4_holdout
contains compounds of the test set.
'''

'''
The standardization strategy is different here. Earlier, for both train and test sets, we used features from the
df_all_3_name_label_standardized matrix which was the entire df_all_3 standardized. This approach was used
to ensure we have the same standardization of features for all cross validation folds. For testing on the holdout
set here, we standardize the train set separately from the test set to ensure no leakage occurs from the test
set being standardized in the same set as the train set. We do this by taking feature values directly from
df_all_3 and standardizing after the train/test split.
'''


# RNA compounds to exclude from the training set
test_set_compounds = X_all_4_holdout[:, 0].tolist()
test_set_RNA_compounds = [item for item in test_set_compounds if item[0:9] != "BindingDB"]
test_set_RNA_compounds_expanded = []
for item in test_set_RNA_compounds:
    test_set_RNA_compounds_expanded.append(item)
    item_expanded = ROBIN_Augment_dict[item]
    for thing in item_expanded:
        test_set_RNA_compounds_expanded.append(thing)
test_set_RNA_compounds_expanded_set = list(set(test_set_RNA_compounds_expanded))

# BindingDB compounds to exclude from the training set
test_set_BindingDB = [item for item in test_set_compounds if item[0:9] == "BindingDB"]

# BindingDB and RNA binders to be excluded from the train set combined into one set
test_set_RNA_expanded_and_BindingDB = test_set_BindingDB + test_set_RNA_compounds_expanded_set

# Remove from df_all_3_name_label_standardized_train BindingDB compounds from the test set
# Also remove from df_all_3_name_label_standardized_train tes set ROBIN RNA binders and their analogues
df_all_3_train = df_all_3.loc[~df_all_3['name'].isin(test_set_RNA_expanded_and_BindingDB)]
# df_all_3_name_label_standardized_train is the train set

# Print shape of the train set and the number of protein and RNA binders within it
print("Shape of the augmented train set:")
print(df_all_3_train.shape)
y_all_3_train_labels_list = df_all_3_train.values[:, -1].tolist()
y_all_3_train_labels_list = [1 if x == "RNA Binding" else x for x in y_all_3_train_labels_list]
y_all_3_train_labels_list = [0 if x == "Protein Binding" else x for x in y_all_3_train_labels_list]
print("Number of protein binders:")
print(int(len(y_all_3_train_labels_list) - sum(y_all_3_train_labels_list)))
print("Number of RNA binders:")
print(int(sum(y_all_3_train_labels_list)))
print()

# Extract from df_all_3_name_label_standardized the test set compounds belonging to BindingDB and ROBIN RNA binders
test_set_RNA_and_BindingDB = test_set_RNA_compounds + test_set_BindingDB
df_all_3_test = df_all_3.loc[df_all_3['name'].isin(test_set_RNA_and_BindingDB)]
df_all_3_test.drop_duplicates(subset=['name'], keep="first", inplace=True)
# df_all_3_name_label_standardized_test is the test set
# Test set compounds are extracted again from df_all_3_name_label_standardized to ensure standardization of train and test sets are the same

# Print shape of the test set and the number of protein and RNA binders within it
print("Shape of the test set:")
print(df_all_3_test.shape)
y_all_3_test_labels_list = df_all_3_test.values[:, -1].tolist()
y_all_3_test_labels_list = [1 if x == "RNA Binding" else x for x in y_all_3_test_labels_list]
y_all_3_test_labels_list = [0 if x == "Protein Binding" else x for x in y_all_3_test_labels_list]
print("Number of protein binders:")
print(int(len(y_all_3_test_labels_list) - sum(y_all_3_test_labels_list)))
print("Number of RNA binders:")
print(int(sum(y_all_3_test_labels_list)))
print()


# Prepare the X_train matrix and standardize as described earlier
X_3_all_Augmented_train = df_all_3_train.values[:, 1:-1]
X_3_all_Augmented_train = X_3_all_Augmented_train.astype('float64')
sc = StandardScaler()
sc.fit(X_3_all_Augmented_train)
X_3_all_Augmented_train = sc.transform(X_3_all_Augmented_train)
print("Shape of X_3_all_Augmented_train matrix:")
print(X_3_all_Augmented_train.shape)
print()

# Prepare the X_test matrix and standardize as described earlier
X_3_all_Augmented_test = df_all_3_test.values[:, 1:-1]
X_3_all_Augmented_test = X_3_all_Augmented_test.astype('float64')
X_3_all_Augmented_test = sc.transform(X_3_all_Augmented_test)
print("Shape of X_3_all_Augmented_test matrix:")
print(X_3_all_Augmented_test.shape)
print()


# Prepare the y_train and y_test matrices
y_3_all_Augmented_train = y_all_3_train_labels_list
y_3_all_Augmented_test = y_all_3_test_labels_list

# Adjust class weights for the MLP model
class_weights = compute_class_weight(class_weight="balanced",
                                        classes=np.unique(y_3_all_Augmented_train),
                                        y=y_3_all_Augmented_train)
class_weights = dict(zip(np.unique(y_3_all_Augmented_train), class_weights))
print("Class_weights:")
print(class_weights)

# Fit MLP model on the train set
MLP_fit = MLP_model.fit(X_3_all_Augmented_train, y_3_all_Augmented_train, class_weight=class_weights)

# Calculate probabilities for the test set
prediction_probs = MLP_model.predict_proba(X_3_all_Augmented_test)[:, 1]

# Calculate classes for the test set
prediction_classes = MLP_model.predict(X_3_all_Augmented_test)

# Calculate confusion matrix
cm = confusion_matrix(y_3_all_Augmented_test, prediction_classes)
print("Confusion matrix on the holdout set:")
print(cm)
print()

# Calculate average precision score
AUPRC_score_holdout = average_precision_score(y_3_all_Augmented_test, prediction_probs)
print("AUPRC score on the holdout set:")
print(round(AUPRC_score_holdout, 2))
print()

# Calculate F1 score
f1_score_holdout = f1_score(y_3_all_Augmented_test, prediction_classes)
print("F1 score on the holdout set:")
print(round(f1_score_holdout, 2))
print()

# Calculate AUROC score
AUROC_score_holdout = roc_auc_score(y_3_all_Augmented_test, prediction_probs)
print("AUROC score on the holdout set:")
print(round(AUROC_score_holdout, 2))
print()

# Calculate Cohen's kappa
cohen_kappa_score_holdout = cohen_kappa_score(y_3_all_Augmented_test, prediction_classes)
print("Cohen’s kappa score on the holdout set:")
print(round(cohen_kappa_score_holdout, 2))
print()

# Calculate Matthews correlation coefficient (MCC)
matthews_corrcoef_holdout = matthews_corrcoef(y_3_all_Augmented_test, prediction_classes)
print("Matthews correlation coefficient on the holdout set:")
print(round(matthews_corrcoef_holdout, 2))
print()

## Test MLP on four RNA binders and four protein binders from the literature

In [ ]:
# Extract df_all_3 column names and their median feature value as a dictionary to fill nan values of the test matrix

df_all_3_values = df_all_3.iloc[:, 1:-1] 
df_all_3_features = df_all_3_values.columns.tolist()
d_df_all_3_feature_medians = {}
for item in df_all_3_features:
    d_df_all_3_feature_medians[item] = df_all_3_values[item].median()
d_df_all_3_feature_medians

In [ ]:
# Test the performance of the neural net on several cases from the scientific literature

# Read in Mordred features of the test compounds
df_Mordred_extra = pd.read_csv(folder_dir + "/data/Mordred_files/Mordred_Test_Compounds_3D.csv")
# Take the compound names as a list as they will be used later
Mordred_extra_names = df_Mordred_extra["name"].tolist()
# Remove the name column
df_Mordred_extra_no_name = df_Mordred_extra.iloc[:, 1:]
# Only keep features of test compounds that also exist in df_all_3
features = df_all_3.columns.tolist()[1:-1]
for item in df_Mordred_extra_no_name:
    if item not in features:
        df_Mordred_extra_no_name = df_Mordred_extra_no_name.drop(item, axis=1)

# Replace infinity in the feature values by nan values
df_Mordred_extra_no_name.replace([np.inf, -np.inf], np.nan, inplace=True)     

# Fill nan values of the test matrix with median of df_all_3 columns
for i in range(8):
    for item in features:
        if np.isnan(df_Mordred_extra_no_name.at[i, item]) == True:
            df_Mordred_extra_no_name.at[i, item] = d_df_all_3_feature_medians[item]
            
df_Mordred_extra_no_name

In [ ]:
# Perform standardization on the train(the entire df_all_3 dataset) and test (12 compounds) sets.

# Import standardization package
from sklearn.preprocessing import StandardScaler

# remove name column from df_all_3
df_all_3_no_name = df_all_3.iloc[:, 1:]

# Convert both df_all_3 and test dataframes to numpy arrays for ML
X_all_3 = df_all_3_no_name.values[:, :-1]
X_extra_test = df_Mordred_extra_no_name.values
y_all_3 = df_all_3_no_name.iloc[:, -1].tolist()

#print shapes of X_all_3, y_all_3, and X_extra_test
print (X_all_3.shape)
print (X_extra_test.shape)
print (len(y_all_3))
print ()

# Standardize X_all_3
X_all_3 = X_all_3.astype('float64')
sc = StandardScaler()
sc.fit(X_all_3)
X_all_3 = sc.transform(X_all_3)

# Standardize X_extra_test using the X_all_3 scaling
# Train and test sets are stadardized separately again to prevent data leakage
X_extra_test = sc.transform(X_extra_test)

# Prepare y_all_3
y_all_3 = [1 if x=="RNA Binding" else x for x in y_all_3]
y_all_3 = [0 if x=="Protein Binding" else x for x in y_all_3]
y_all_3 = np.array(y_all_3)
print ("done")

In [ ]:
# The neural network MLP model (same as before) is repeated here for reference.

# Import the neural network packages
import tensorflow as tf
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import Sequential
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# Define the neural network MLP model and wrap it in the scikitlearn pipeline for easier use

# Define neural network as function
def create_model():

    # create model
    model = Sequential()

    # Define number of features
    number_of_features = 1665

    # Hidden layers
    model.add(layers.Dense(units=200, activation='relu', kernel_regularizer=regularizers.l2(0.00001),
                           input_shape=(number_of_features,)))

    model.add(layers.Dense(units=200, activation='relu', kernel_regularizer=regularizers.l2(0.00001)))

    # Output layer
    model.add(layers.Dense(units=1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy',  # Cross-entropy
                  optimizer='Adam',  # Use Adam optimizer
                  metrics=['AUC'])
    return model


# Wrap within the scikitlearn framework
MLP_model = KerasClassifier(create_model, epochs=7, batch_size=2048, verbose=1)

In [ ]:
# Train the MLP model on X_all_3 and y_all_3 and test on X_extra_test

# Set random seed for numpy and tensorflow
RANDOM_SEED = 5
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Make sure the slight imbalance in the training set is accounted for in the neural network
class_weights = compute_class_weight(class_weight = "balanced",
                                        classes = np.unique(y_all_3),
                                        y = y_all_3)
class_weights = dict(zip(np.unique(y_all_3), class_weights))
print (class_weights)


# Fit the MLP model to the training set
MLP_fit = MLP_model.fit(X_all_3, y_all_3)

# Predict probabilities for X_extra_test
prediction_extra = MLP_model.predict_proba(X_extra_test)

# Print compound name followed by the probability predicted by model
for i in range(len(prediction_extra)):
    print (Mordred_extra_names[i])
    print (round(prediction_extra[i][1]*100, 1))
    print ()

## SHAP Analysis

In [ ]:
#Run the model on the entire dataset in preparation for the SHAP analysis

model = Sequential()
number_of_features = 1665

model.add(layers.Dense(units=200, activation='relu', kernel_regularizer=regularizers.l2(0.00001),input_shape=(number_of_features,)))

model.add(layers.Dense(units=200, activation='relu', kernel_regularizer=regularizers.l2(0.00001)))

model.add(layers.Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='Adam', 
                  metrics=['AUC'])

history = model.fit(X_all_3, y_all_3, epochs=7, batch_size=2048, verbose=1) 


In [ ]:
#Perform SHAP to interpret the neural network model

# Import shap package
import shap

#initialize
shap.initjs()

#define the background 
background = X_all_3[np.random.choice(X_all_3.shape[0], 1000, replace=False)]

#use the deepexplainer package within SHAP tailored for use in neural networks
e = shap.DeepExplainer(model, background)

#calculate SHAP values
shap_values = e.shap_values(X_all_3)

#print SHAP values
print(shap_values)

In [ ]:
# Save SHAP values to CSV as the SHAP code takes a long time to run
df_shap_save = pd.DataFrame(shap_values[0])
df_shap_save.to_csv(folder_dir + "/ML_CSVs/SHAP_values.csv")

In [ ]:
# Read back in the saved CSV containing the SHAP values
df_SHAP_array = pd.read_csv(folder_dir + "/ML_CSVs/Shap_values.csv")
df_SHAP_array = df_SHAP_array.drop('Unnamed: 0', axis=1)
df_SHAP_array = df_SHAP_array.to_numpy()
print(df_SHAP_array.shape)
df_SHAP_array

In [ ]:
# Draw the SHAP beeswarm plot

# Set plot font to Halvetica
plt.rc('font', family='Helvetica') 

df_SHAP = df_all_3.drop('library', axis=1)
df_SHAP = df_SHAP.drop('name', axis=1)
shap.summary_plot(df_SHAP_array, df_SHAP, show=False)
plt.savefig(folder_dir + '/figures/SHAP_Beeswarm_plot.png', bbox_inches="tight" ,dpi=600)

In [ ]:
# Generate the barplot for the SHAP beeswarm plot

shap.summary_plot(shap_values, df_SHAP, show=False)
plt.savefig(folder_dir + '/figures/SHAP_Barplot.png', bbox_inches="tight" ,dpi=600)